In [2]:
import pandas

In [3]:
df = pandas.read_feather(f"dataset_4.feather")
df

,instance,problem,correct_program_lines,correct_program,incorrect_program_lines,incorrect_program,correction,n_mutations,missing_lines,n_mising_lines,fl,line_scores
0,mooshak/01/A_0_1.lp,A,"[color((1..k))., node(N) :- e(_, N)., node(N) ...","color((1..k)).\nnode(N) :- e(_, N).\nnode(N) :...","[node(N) :- e(_, N)., :- e(N, M), assign(N, C...","node(N) :- e(_, N).\n :- e(N, M), assign(N, C)...","[color((1..k))., node(N) :- e(N, _)., 1 { assi...",-1,True,3,[],"[0.0, 0.0]"
1,mooshak/01/A_0_1.lp,A,"[color((1..k))., node(N) :- e(_, N)., node(N) ...","color((1..k)).\nnode(N) :- e(_, N).\nnode(N) :...","[node(N) :- e(_, N)., node(N) :- e(N, _)., 1 {...","node(N) :- e(_, N).\nnode(N) :- e(N, _).\n1 { ...","[color((1..k))., :- e(N, M), assign(N, C), as...",-1,True,2,[],"[0.0, 0.0, 0.0]"
2,mooshak/01/A_0_1.lp,A,"[color((1..k))., node(N) :- e(_, N)., node(N) ...","color((1..k)).\nnode(N) :- e(_, N).\nnode(N) :...","[node(N) :- e(N, _)., 1 { assign(N, C) : color...","node(N) :- e(N, _).\n1 { assign(N, C) : color(...","[color((1..k))., node(N) :- e(_, N).]",-1,True,2,[],"[0.0, 0.0, 0.0]"
3,mooshak/01/A_0_1.lp,A,"[color((1..k))., node(N) :- e(_, N)., node(N) ...","color((1..k)).\nnode(N) :- e(_, N).\nnode(N) :...","[color((1..k))., node(N) :- e(_, N)., 1 { assi...","color((1..k)).\nnode(N) :- e(_, N).\n1 { assig...","[node(N) :- e(N, _).]",-1,True,1,[],"[0.0, 0.0, 0.0, 0.0]"
4,mooshak/01/A_0_1.lp,A,"[color((1..k))., node(N) :- e(_, N)., node(N) ...","color((1..k)).\nnode(N) :- e(_, N).\nnode(N) :...","[node(N) :- e(_, N)., node(N) :- e(N, _)., 1 {...","node(N) :- e(_, N).\nnode(N) :- e(N, _).\n1 { ...",[color((1..k)).],-1,True,1,[],"[0.0, 0.0, 0.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
19815,mooshak/02/E_9_0.lp,E,"[1 { set(X, a); set(X, b) } 1 :- vertex(X)., ...","1 { set(X, a); set(X, b) } 1 :- vertex(X).\n :...","[1 { set(X, a); set(X, b) } 1 :- vertex(X)., b...","1 { set(X, a); set(X, b) } 1 :- vertex(X).\nb ...","[ :- edge(A, B), set(A, X), set(B, X).]",6,False,0,"[1, 2]","[0.0, 1.0, 1.0]"
19816,mooshak/02/E_9_0.lp,E,"[1 { set(X, a); set(X, b) } 1 :- vertex(X)., ...","1 { set(X, a); set(X, b) } 1 :- vertex(X).\n :...","[1 { set(X, a); set(X, b) } 1 :- vertex(X)., b...","1 { set(X, a); set(X, b) } 1 :- vertex(X).\nb ...","[ :- edge(A, B), set(A, X), set(B, X).]",6,False,0,"[1, 2]","[0.0, 1.0, 1.0]"
19817,mooshak/02/E_9_0.lp,E,"[1 { set(X, a); set(X, b) } 1 :- vertex(X)., ...","1 { set(X, a); set(X, b) } 1 :- vertex(X).\n :...","[1 { set(X, a); set(X, b) } 1 :- vertex(X)., b...","1 { set(X, a); set(X, b) } 1 :- vertex(X).\nb ...","[ :- edge(A, B), set(A, X), set(B, X).]",7,False,0,"[1, 2]","[0.0, 1.0, 1.0]"
19818,mooshak/02/E_9_0.lp,E,"[1 { set(X, a); set(X, b) } 1 :- vertex(X)., ...","1 { set(X, a); set(X, b) } 1 :- vertex(X).\n :...","[1 { set(X, a); set(X, b) } 1 :- vertex(X)., I...","1 { set(X, a); set(X, b) } 1 :- vertex(X).\nI0...","[ :- edge(A, B), set(A, X), set(B, X).]",7,False,0,"[1, 2]","[0.0, 1.0, 1.0]"


In [ ]:
df["missing_lines"]